# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

# Setting up API and user information

In [ ]:
import requests
import pandas as pd

user_key = 'Input your User Key'

root_url = f'https://aifapbt.fin.cloud.ainode.ai/{user_key}/'

# health check

In [ ]:
requests.get(root_url).json()

# upload to main server


### strategy


In [ ]:
# Upload strategy file
endpoint = 'upload/strategy/'
url = root_url + endpoint

file_path = "/content/drive/MyDrive/~~~" # Input a path that contains a proven your strategy(python file)

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, files=files)

print("📂 Upload Response:", response.json())

In [ ]:
# Check strategy file upload
endpoint = 'upload/check/strategy/'
url = root_url + endpoint

params = {"strategy_name": "Input your strategy name"} # example : "Momentum.py" --> "Momentum"

response = requests.get(url, params=params)
print(response.json()['content'])

In [ ]:
# Check all uploaded strategy files
endpoint = 'upload/check/all-files/strategy/'
url = root_url + endpoint

response = requests.get(url)
print(response.json())

In [ ]:
# # Delete uploaded strategy settings file
# endpoint = 'upload/delete/strategy/'
# url = root_url + endpoint

# params = {"strategy_name": "Input your strategy name"} # example : "Momentum.py" --> "Momentum"

# # Sending a DELETE request
# response = requests.delete(url, params=params)
# print(response.json())

### config

In [ ]:
# Upload strategy file
endpoint = 'upload/config/'
url = root_url + endpoint

file_path = "/content/drive/MyDrive/finplatform/multi_period_momentum_demo_config.py" # use demo version for example test

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, files=files)

print("📂 Upload Response:", response.json())

In [ ]:
# Check strategy file upload
endpoint = 'upload/check/config/'
url = root_url + endpoint

params = {"config_name": "Input your config name"} # example : "*_config.py" --> "*_config"

response = requests.get(url, params=params)
print(response.json()['content'])

In [ ]:
# Check all uploaded config files
endpoint = 'upload/check/all-files/config/'
url = root_url + endpoint

response = requests.get(url)
print(response.json())

In [ ]:
# # Delete uploaded strategy settings file
# endpoint = 'upload/config/delete/'
# url = root_url + endpoint

# params = {"config_name": "Input your config name"} # example : "*_config.py" --> "*_config"

# # Sending a DELETE request
# response = requests.delete(url, params=params)
# print(response.json())

# run auto trade

In [ ]:
# run-system
endpoint = 'command/run-system'
url = root_url + endpoint

data = {
    "strategy_name": "Input your strategy name", # example : "Momentum.py" --> "Momentum"
    "method": "rebalancing" # Select : "general" , "rebalancing" , "volatility targeting"
}

response = requests.post(url, json=data)

try:
  session_id = response.json()['session_id']
  print('session id :',response.json()['session_id'])
  print('dashboard :',response.json()['dashboard_url'])
except:
  print('Error :',response.json()['message']['message'])

  session_id = response.json()['message']['session_id']
  print('session id :',session_id)

  print('dashboard :',response.json()['message']['dashboard_url'])

In [ ]:
# !python {your path for 'log_viewer.py'} --user_key {user_key} --session_id {session_id} --save_log true/false
!python /content/drive/MyDrive/~~~/log_viewer.py --user_key {user_key} --session_id {session_id} --save_log true

# terminate

In [ ]:
# terminate process
# You must execute terminate to force liquidation of the current process.
endpoint = 'command/terminate/'
url = root_url + endpoint

params = {"session_id": f"{session_id}"}

response = requests.get(url, params=params)
print(response.json())

# Confirm position holding after Terminating

In [ ]:
def all_positions(user_key, productType, marginCoin):
    url = f"https://bitgettrader.fin.cloud.ainode.ai/{user_key}/future/position/all-positions"

    payload = {
        "user_key": user_key,
        "all_positions": {
            "productType": productType,
            'marginCoin': marginCoin.upper()
        }
    }

    response = requests.post(url, json=payload)
    data = response.json()['data']
    df = pd.DataFrame(data)
    if df.empty:
        print('Everything has been liquidated')
        pass
    return df


productType = 'usdt-futures' # use demo version for test. Originally 'usdt-futures'
marginCoin = 'usdt' # use demo version for test. Originally 'usdt'
df = all_positions(user_key, productType, marginCoin)
df